<div style=" border-bottom: 8px solid #e3f56c; overflow: hidden; border-radius: 10px; height: 60px; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">XGBoost</span></b>
  </div>
</div>

<div class="alert" style="background-color: #FEDAD5; border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
  <h5 style="font-size: 20px; margin-bottom: 10px;">
    <strong> Contents </strong>
  </h5>
<hr>
  <p><font size="3" face="Arial" font-size="large">
  <ul type="square">
      
- `DMatrix` for data preprocessing  
- Two main types of model training in XGBoost  
- Important model hyperparameters  
- Hyperparameter tuning  
- `LearningRateScheduler`  
- Custom metrics from `scikit-learn`  
- Conclusions and summary  


  </ul>
  </font></p>


</div>

<div>

<img src='https://cdn-images-1.medium.com/max/800/1*l4PN8hyAO4fMLxUbIxcETA.png' align="left" width=65% height=65% />
</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

`XGBoost` was developed as a research project at the University of Washington. Tianqi Chen and Carlos Guestrin presented their work at the SIGKDD conference in 2016, making a big impact in the machine learning world. Since its release, this algorithm has not only dominated Kaggle competitions but has also powered numerous industrial applications.
<div>

<div class="alert alert-info">

### **Some key features:**
</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* Rich set of regularization parameters (`lambda` / `gamma` / `alpha`)
* Considered one of the fastest and least resource-intensive boosting algorithms (though debatable)
* Built-in handling of missing values and categorical features (via `one-hot encoding`) (`enable_categorical = True` / `df[cat_col].astype('category')`)
* Native support for metrics from **scikit-learn** as custom metrics
<div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* Native support for `Dask` and `Spark`
* Compatible with popular cloud platforms and supports distributed training across clusters or multiple machines
* Supports most major programming languages
* Integrates well with other frameworks and services (**Optuna**, **Weights & Biases**, etc.)
<div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

**Boosting Implementation Details (see the diagram):**

* **Parallelization:** In `XGBoost`, tree construction is parallelized. This is made possible by rearranging the nested loops used to compute split points: the outer loop iterates over tree leaves, and the inner one over features. Swapping the loops allows efficient parallel computation. Initialization occurs during data loading, followed by sorting with parallel threads, which significantly speeds up training.

* **Tree Pruning (`gamma`):** Unlike traditional GBMs that use negative loss reduction as a split-stopping criterion, `XGBoost` employs `max_depth` and performs *post-pruning* based on a minimum loss reduction (`gamma`). This approach improves computational efficiency.

* **Hardware Optimization:** The algorithm is engineered to maximize hardware utilization by creating internal buffers per thread to store gradient statistics.

* **Regularization:** To prevent overfitting, `XGBoost` applies both L1 (LASSO) and L2 (Ridge) regularization.

* **Handling Missing Values:** `XGBoost` handles sparse data gracefully, learning how to assign missing values based on the direction that minimizes loss.

* **Cross-Validation:** The algorithm includes a built-in method for performing cross-validation during training.

</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

From the user’s perspective, `XGBoost` is quite similar to `LightGBM`: it uses a `booster` object, has a dedicated **sklearn API**, supports `dart`, `callbacks`, `scheduler`, etc. However, there are some key differences worth exploring.

</div>

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

!pip install xgboost
</div>

In [2]:
import xgboost as xgb

import numpy as np
import pandas as pd

import warnings; warnings.filterwarnings("ignore")
from classes import Paths

In [3]:
paths = Paths()
path = paths.quickstart_train
df = pd.read_csv(path)
print(df.shape)
df.head(10)

(2337, 17)


,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.10,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.00,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.10,102.382857,2.513319,174,173
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,32.22,engine_fuel,3.880920,1.651883e+07,0.10,172.793237,-5.029476,174,170
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,27.51,engine_fuel,4.181149,1.398317e+07,0.10,203.462289,-14.260456,174,171
5,b12101843B,Skoda Rapid,economy,petrol,2.36,2013,42176,2018,48.99,engine_ignition,4.351782,1.085589e+07,0.10,180.886289,-18.221832,174,173
6,Q-9368117S,Nissan Qashqai,standart,petrol,5.32,2012,24611,2014,54.72,engine_overheat,4.392126,8.343280e+06,0.10,174.984786,12.321364,174,167
7,O-2124190y,Tesla Model 3,premium,electro,3.90,2017,116872,2019,50.40,gear_stick,4.712356,9.793288e+06,0.10,95.890736,-8.939366,174,139
8,h16895544p,Kia Sportage,standart,petrol,3.50,2014,56384,2017,33.59,gear_stick,4.507759,1.644405e+07,0.32,101.798615,-1.164690,174,170
9,K77009462l,Smart ForFour,economy,petrol,4.56,2013,41309,2018,39.04,gear_stick,4.376839,6.975742e+06,0.10,125.254983,3.769684,174,173


In [5]:
drop_cols = ['car_id', 'target_reg', 'target_class']
cat_cols = ['car_type', 'fuel_type', 'model']

X = df.drop(drop_cols, axis=1)
y = df['target_reg']

<div class="alert alert-warning">

### **`DMatrix` for Data Preprocessing**
</div>

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* `CatBoost` -> `Pool`
* `LightGBM` -> `Dataset`
* `XGBoost` -> `DMatrix`
</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

The `DMatrix` class is designed to create a data structure optimized for use in `XGBoost`. It's built for efficient memory usage and fast training. It supports various data formats and includes a handy `missing` parameter to handle missing values on the fly.<br>

Let's go over its key parameters, especially those not typically found in other boosting libraries:

* `data` – training data
* `label` – target values
* `nthread` – number of parallel threads to use when loading data (`-1` uses all available threads)
* `missing` (default: `np.nan`) – specify the value to treat as missing
* `enable_categorical` – automatically detects categorical features if the DataFrame columns are pre-converted to `category` type.<br>
* As with `LightGBM`, string-based categorical features must be converted to integers beforehand (i.e., apply label encoding).<br>

</div>


<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* It also provides a useful method `save_binary(fname, silent=True)` – this allows saving the dataset in a binary format, reducing memory usage and speeding up future experiments. <br>

* ⚠️ Notably, there is a special class called `QuantileDMatrix`, which creates a quantile-based dataset directly from the data source. This helps conserve memory by avoiding intermediate data storage. It introduces a few extra parameters:
    * `max_bin` – controls the number of bins for quantization (must match the model's `max_bin` parameter)
    * `ref` – used when creating a validation dataset; refers to a training `QuantileDMatrix` to reuse quantile information and prevent data leakage.
</div>

In [6]:
from sklearn.model_selection import train_test_split

# let's change cat features to 'category' type
for col in cat_cols:
    X[col] = X[col].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
dtrain = xgb.DMatrix(X_train,
                     label=y_train,
                     nthread=-1,
                     enable_categorical=True,)

dtest = xgb.DMatrix(X_test,
                    y_test,
                    nthread=-1,
                    enable_categorical=True)